# Comparing the various table sampling approaches


With the `fake_*()` function, we need to pick a random line inside the fake table. We need to do it fast.

## Requirements

This notebook requires a `createdb pg_anonymizer_dev_notes --owner jupyter ` owned by a `jupyter` role in a PostgreSQL database

```bash
$ createuser jupyter --password
$ createdb pg_anonymizer_dev_notes --owner jupyter  
```

In [61]:
%reload_ext sql
%sql postgresql://jupyter@localhost/pg_anonymizer_dev_notes



## Create a test table





In [64]:
%%sql

CREATE SCHEMA IF NOT EXISTS anon;

DROP TABLE IF EXISTS anon.email;
CREATE TABLE anon.email(
    oid SERIAL,
    address TEXT
);

INSERT INTO anon.email(address)
 SELECT md5(random()::TEXT) 
 FROM generate_series(1,1000);

CREATE INDEX ON anon.email(oid); 
CLUSTER anon.email;

 * postgresql://jupyter@localhost/pg_anonymizer_dev_notes
Done.
Done.
Done.
1000 rows affected.
Done.
(psycopg2.errors.UndefinedObject) there is no previously clustered index for table "email"

[SQL: CLUSTER anon.email;]
(Background on this error at: http://sqlalche.me/e/13/f405)


## With tsm_system_rows

This is first implementation was based on tsm_system_rows.

This method is fast but it has a "clustering effect" on small datasets

https://www.postgresql.org/docs/current/tsm-system-rows.html

In [65]:
%%sql
-- CREATE EXTENSION IF NOT EXISTS tsm_system_rows SCHEMA anon;

CREATE OR REPLACE FUNCTION anon.fake_email_tsm_system_rows()
RETURNS TEXT AS $$                                                                                                                                                        
  SELECT address 
  FROM anon.email
  TABLESAMPLE anon.system_rows(1);
$$
LANGUAGE SQL VOLATILE SECURITY INVOKER SET search_path='';
 

 * postgresql://jupyter@localhost/pg_anonymizer_dev_notes
Done.


[]

In [66]:
%%time
%%sql
SELECT count(DISTINCT anon.fake_email_tsm_system_rows()) 
FROM generate_series(1,100000); 



 * postgresql://jupyter@localhost/pg_anonymizer_dev_notes
1 rows affected.
CPU times: user 4.05 ms, sys: 5.45 ms, total: 9.5 ms
Wall time: 4.93 s


count
9


## With tsm_system

This is the standard sampling method.

In [67]:
%%sql                                                                                                                 
CREATE OR REPLACE FUNCTION anon.fake_email_tsm_system()                                                                                                                                  
RETURNS TEXT AS $$
    SELECT address
    FROM anon.email
    TABLESAMPLE SYSTEM(50) LIMIT 1;
$$
LANGUAGE SQL VOLATILE SECURITY INVOKER SET search_path='';     

 * postgresql://jupyter@localhost/pg_anonymizer_dev_notes
Done.


[]

In [68]:
%%time
%%sql
SELECT count(DISTINCT anon.fake_email_tsm_system()) 
FROM generate_series(1,100000); 

 * postgresql://jupyter@localhost/pg_anonymizer_dev_notes
1 rows affected.
CPU times: user 0 ns, sys: 9.12 ms, total: 9.12 ms
Wall time: 5 s


count
9


## With random_and_max

This is the classique way to do this

In [69]:

%%sql

CREATE OR REPLACE FUNCTION anon.fake_email_random_and_max()                                                                                                                                  
RETURNS TEXT AS $$
    WITH random AS (
        SELECT (random()*max(oid)+1)::INT AS oid
        FROM anon.email
    )
    SELECT address
    FROM anon.email e
    JOIN random r ON e.oid=r.oid                                                                                                                                                           ;                                                                                                               
$$
LANGUAGE SQL VOLATILE SECURITY INVOKER SET search_path='';                                                                                                                     
                                                              

 * postgresql://jupyter@localhost/pg_anonymizer_dev_notes
Done.


[]

In [70]:
%%time
%%sql
SELECT count(DISTINCT anon.fake_email_random_and_max()) 
FROM generate_series(1,100000); 

 * postgresql://jupyter@localhost/pg_anonymizer_dev_notes
1 rows affected.
CPU times: user 8.33 ms, sys: 1.21 ms, total: 9.54 ms
Wall time: 13.5 s


count
1000


## With Last Value

Try to avoid the `max()` aggregate function by using the sequence.

**For now, this is the best tradeoff**

In [71]:
%%sql

CREATE OR REPLACE FUNCTION anon.fake_email_last_value()
RETURNS TEXT AS $$
    WITH random AS (
        SELECT (random()*last_value+1)::INT AS oid
        FROM anon.email_oid_seq
    )
    SELECT address
    FROM anon.email e
    JOIN random r ON e.oid=r.oid
$$
LANGUAGE SQL VOLATILE SECURITY INVOKER SET search_path=''; 

 * postgresql://jupyter@localhost/pg_anonymizer_dev_notes
Done.


[]

In [74]:
%%time
%%sql
SELECT count(DISTINCT anon.fake_email_last_value()) 
FROM generate_series(1,100000); 

 * postgresql://jupyter@localhost/pg_anonymizer_dev_notes
1 rows affected.
CPU times: user 8.63 ms, sys: 805 µs, total: 9.44 ms
Wall time: 9.06 s


count
1000


## With ORDER BY

The most naïve way to do it ! Even if the table is clustered on the oid column. The performances are horrible.

In [78]:
%%sql

CREATE OR REPLACE FUNCTION anon.fake_email_order_by()
RETURNS TEXT AS $$
    SELECT address
    FROM anon.email
    ORDER BY random()
    LIMIT 1;
$$
LANGUAGE SQL VOLATILE SECURITY INVOKER SET search_path=''; 


 * postgresql://jupyter@localhost/pg_anonymizer_dev_notes
Done.


[]

In [79]:
%%time
%%sql
SELECT count(DISTINCT anon.fake_email_order_by()) 
FROM generate_series(1,100000); 

 * postgresql://jupyter@localhost/pg_anonymizer_dev_notes
1 rows affected.
CPU times: user 8.24 ms, sys: 5.34 ms, total: 13.6 ms
Wall time: 1min 27s


count
1000


## With Offset

This is a bit better than ORDER BY, but still a bad idea

In [75]:
%%sql

CREATE OR REPLACE FUNCTION anon.fake_email_offset()
RETURNS TEXT AS $$
    SELECT address
    FROM anon.email
    OFFSET floor(random()*(SELECT last_value FROM anon.email_oid_seq)+1)
    LIMIT 1;
$$
LANGUAGE SQL VOLATILE SECURITY INVOKER SET search_path=''; 


 * postgresql://jupyter@localhost/pg_anonymizer_dev_notes
Done.


[]

In [76]:
%%time
%%sql
SELECT count(DISTINCT anon.fake_email_offset()) 
FROM generate_series(1,100000); 

 * postgresql://jupyter@localhost/pg_anonymizer_dev_notes
1 rows affected.
CPU times: user 3.83 ms, sys: 7.23 ms, total: 11.1 ms
Wall time: 21.2 s


count
999


## Links                                                                                                                                                                      
                                                                                                                                                                               
* <https://stackoverflow.com/questions/5297396/quick-random-row-selection-in-postgres/5298588#5298588>                                                                         
* <https://blog.2ndquadrant.com/tablesample-in-postgresql-9-5-2/>                                                                                                              
* <http://web.archive.org/web/20080214235745/http://www.powerpostgresql.com/Random_Aggregate>                                                                                  
* <https://www.postgresql.org/docs/current/static/tsm-system-rows.html>         